# Code

Hoe schrijven we code?

Mijn advies: **Value Objects + Services** -> scheiding tussen data en gedrag+configuratie

* design volgt natuurlijk
* gemakkelijk aan te passen

9 van de 10 keer de beste keuze

# Tegenovergestelde
 
fat model -> data en gedrag in hetzelfde object

* Veel design van te voren nodig
* Stateful / mutable -> beperkt functionaliteit
* Moeilijk aan te passen
* Verleiding tot globals, singletons, statics.

# Value Objects

= DTO's (Data Transfer Objects)

Gebruikt voor:

- input
- output
- communicatie tussen interne services (controllers, services, repositories)

In [2]:
from dataclasses import dataclass
from datetime import datetime, timedelta

@dataclass
class SensorSubmission:
    start: datetime
    end: datetime
    max_blinde_vering_mm: float


sensor_submission = SensorSubmission(
    start=datetime.fromisoformat('2024-07-18 00:00:00.283+02:00'),
    end=datetime.fromisoformat('2024-07-19 00:00:00.283+02:00'),
    max_blinde_vering_mm=55,
)

sensor_submission

SensorSubmission(start=datetime.datetime(2024, 7, 18, 0, 0, 0, 283000, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), end=datetime.datetime(2024, 7, 19, 0, 0, 0, 283000, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), max_blinde_vering_mm=55)

# Normalisatie

Single source of truth

In [5]:
from datetime import datetime, timedelta

# fout
@dataclass
class SensorSubmission:
    start: datetime
    end: datetime
    duration: timedelta # kan afgeleid worden van start en end
    max_blinde_vering_mm: float
    
# goed
@dataclass
class SensorSubmission:
    start: datetime
    end: datetime
    max_blinde_vering_mm: float

    def duration(self) -> timedelta:
        return self.end - self.start

# fout
@dataclass
class SensorSubmission:
    blinde_vering_per_passage_mm: list[float]
    max_blinde_vering_mm: float # kan afgeleid worden van blinde_vering_per_passage_mm
    
# goed
@dataclass
class SensorSubmission:
    blinde_vering_per_passage_mm: list[float]
    
    def max_blinde_vering_mm(self) -> float:
        return max(self.blinde_vering_per_passage_mm)

Naamgeving: onderzoek de naamgeving gehanteerd binnen het domein (bedrijfstak) gebruik deze waar van toepassing.

# Services

Logica, configuratie en dependencies

Veel design patterns zijn hier van toepassing

In [ ]:
class SensorSubmissionRepository:
    # Constructor injection
    def __init__(self, db_connection):
        self.db_connection = db_connection 

    def save(self, submission: SensorSubmission):
        self.db_connection.execute('''
            INSERT INTO sensor_submissions (start, end, max_blinde_vering_mm)
            VALUES (?, ?, ?)
        ''', (submission.start, submission.end, submission.max_blinde_vering_mm))
            

class SensorSubmissionService:
    def __init__(self, submission_repository: SensorSubmissionRepository, sms_gateway):
        self.submission_repository = submission_repository
        self.sms_gateway = sms_gateway

    def process_sensor_submission(self, submission: SensorSubmission):
        self.repository.save(submission)
        
        if submission.max_blinde_vering_mm > 90:
            self.sms_gateway.send_sms('Blinde vering boven limiet')

Houd code op hetzelfde niveau van abstractie

In [ ]:
# fout
class SensorSubmissionServiceV2:
    def __init__(self, repository: SensorSubmissionRepository, sms_gateway, user_service):
        self.repository = repository
        self.sms_gateway = sms_gateway
        self.user_service = user_service

    def process_sensor_submission(self, submission: SensorSubmission):
        self.repository.save(submission)

        user_profile = self.user_service.fetch_profile()
        user_preferences = self.user_service.fetch_preferences()
        
        should_alert = (user_profile.phone_number is not None 
                        and user_preferences.alerts_enabled
                        and submission.max_blinde_vering_mm > 90)
        
        if should_alert:
            self.sms_gateway.send_sms('Blinde vering boven limiet')

# goed
class SensorSubmissionServiceV2:
    def __init__(self, repository: SensorSubmissionRepository, alert_service: AlertService):
        self.th
        self.repository = repository
        self.alert_service = alert_service

    def process_sensor_submission(self, submission: SensorSubmission):
        # 2 calls op zelfde niveau van abstractie
        self.repository.save(submission)
        self.alert_service.process_sensor_submission(submission)
        

class AlertService:
    def __init__(self, sms_gateway, user_service):
        self.sms_gateway = sms_gateway
        self.user_service = user_service

    def process_sensor_submission(self, submission: SensorSubmission) -> bool:
        user_profile = self.user_service.fetch_profile()
        user_preferences = self.user_service.fetch_preferences()

        should_alert = (user_profile.phone_number is not None
                        and user_preferences.alerts_enabled
                        and submission.max_blinde_vering_mm > 90)

        if should_alert:
            self.sms_gateway.send_sms('Blinde vering boven limiet')
            return True
        
        return False